In [2]:
# Imports
import os 
from predictionguard import PredictionGuard
import pandas as pd 
import json
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
import numpy as np
from torch import nn
from skorch import NeuralNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, auc, RocCurveDisplay, ConfusionMatrixDisplay, accuracy_score, classification_report
import matplotlib.pyplot as plt
from torch.optim import Adam
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import time

In [3]:
# Grief Journey Map
JourneyMap = {
    "1. Immediate Grief, Shock, and Emotion": "Overwhelmed, loss of purpose; shock and trauma emotions (isolation) present and challenging to understand. Individuals may struggle to deal with family responsibilities alone. Surviving Child: Feeling disconnected without guidance and attention from grieving adults.", 

    "2. Navigating Family Relationships": "Experiencing tension between individuals within the family unit; lack of support from family members. Surviving Family Unit: Perception of other family members’ grief experience. Each family member may be at different phases of their grief journey.",

    "3. Learning to Process Grief": "Experiencing grief and learning to process those emotions. Surviving Child: Seeks guidance and acknowledgment of grief; benefit from opportunities to open up and process with kids in similar situations to normalize emotions.", 

    "4. Moments That Matter": "Renewed experience of grief around anniversaries of loss, holidays, and special moments. Surviving Family Unit: Navigating special moments (sports, school achievements, moments that matter).", 

    "5. Feeling Immersed, Connected & Seen": "Finding new purpose and goals to begin moving towards Positive Integration. Surviving Family Unit: Connected to a broader community; support system; not the only person/family experiencing loss.", 

    "6. New Growth & Purpose": "Healthy point in grief journey; feeling capable to help others and a desire to do so. Surviving Family Unit: Ready to give back to the TAPS community through mentorship programs, volunteering at charity drives & events, etc." 
    }

dict = {
    "Stage Number": [int(key.split('.')[0]) for key in JourneyMap.keys()],
    "Stage Name": [key.split('. ', 1)[1] for key in JourneyMap.keys()],
    "Description": JourneyMap.values()
}

JourneyMapDF = pd.DataFrame(dict)
JourneyMapDF.set_index("Stage Number", inplace=True)
JourneyMapDF.head()

,Stage Name,Description
Stage Number,,
1,"Immediate Grief, Shock, and Emotion","Overwhelmed, loss of purpose; shock and trauma..."
2,Navigating Family Relationships,Experiencing tension between individuals withi...
3,Learning to Process Grief,Experiencing grief and learning to process tho...
4,Moments That Matter,Renewed experience of grief around anniversari...
5,"Feeling Immersed, Connected & Seen",Finding new purpose and goals to begin moving ...


In [5]:
# Survey Questions
SurveyQuestions = [
    {
        "question": "Please share all of the reasons you chose to attend the TAPS [SEMINAR NAME]",
        "options": [
            "I had attended a previous Regional or Nationals seminar",
            "To connect with other survivors who share a similar loss",
            "To learn more about resources TAPS has to offer",
            "To learn new tools and information to help me with my grief",
            "To learn more about how to support my adult family members in their grief",
            "To learn more about how to support my child(ren) in their grief",
            "For my child(ren) to attend Good Grief Camp",
            "For my child(ren) to connect with a Military Mentor"
        ],
        "type": "multiple_choice"
    },
    {
        "question": "How did you find out about the TAPS [SEMINAR NAME]?",
        "options": [
            "I found this event while searching for grief resources",
            "I attended a seminar last year and had already marked my calendar!",
            "TAPS invited me to this event via email",
            "My TAPS Survivor Care Team Member invited me",
            "My Peer Mentor or another survivor invited me",
            "Through the TAPS website",
            "Through the TAPS magazine",
            "Through a TAPS Social Media Page"
        ],
        "type": "multiple_choice"
    },
    {
        "question": "This seminar helped me to feel socially connected",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "This seminar taught me new ways to cope with my grief",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "This seminar helped me to better understand my grief",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "This seminar has given me hope for the future",
        "options": ["Disagree", "Neither agree nor disagree", "Agree"],
        "type": "likert"
    },
    {
        "question": "Based on your experience at the TAPS [SEMINAR NAME], how likely are you to continue to connect with TAPS virtually / at in-person programs?",
        "options": [
            "Extremely likely / Very likely",
            "Somewhat likely",
            "Neither likely nor unlikely / Neutral",
            "Somewhat unlikely",
            "Extremely unlikely / Not at all likely"
        ],
        "type": "likelihood"
    },
    {
        "question": "Based on your experience at the TAPS [SEMINAR NAME], how likely are you to continue to bring your child(ren) to connect with TAPS at in-person programs?",
        "options": [
            "Extremely likely / Very likely",
            "Somewhat likely",
            "Neither likely nor unlikely / Neutral",
            "Somewhat unlikely",
            "Extremely unlikely / Not at all likely"
        ],
        "type": "likelihood"
    },
    {
        "question": "I have a positive outlook toward life",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have short and/or long range goals",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I feel all alone",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I can see possibilities in the midst of difficulties",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have faith that gives me comfort",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I feel scared about my future",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I can recall happy/joyful times",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have deep inner strength",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I am able to give and receive care/love",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I have a sense of direction",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I believe that each day has potential",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "I feel my life has value and worth",
        "options": ["Strongly Disagree", "Disagree", "Agree", "Strongly Agree"],
        "type": "likert"
    },
    {
        "question": "Please share with TAPS your favorite moment of the weekend? Did you have a breakthrough moment this weekend you would like to share?",
        "type": "free_text"
    },
    {
        "question": "Please share any additional feedback you have regarding your TAPS Seminar experience.",
        "type": "free_text"
    }
]

In [6]:
# Training Data Import
data = pd.read_csv("examples.csv")


In [18]:
# Function to create prompt with stage-specific examples for free-text questions
def specific_prompt_free_response(stage_number, question_text, data):
    # Filter the data for the specific grief stage
    stage_data = data[data["label"] == stage_number]
    
    # Limit to a few examples to keep the prompt concise
    stage_examples = stage_data.sample(min(5, len(stage_data)))  # Get up to 3 examples

    # Format examples for the prompt
    example_texts = "\n\n".join(
        [
            f"Example {i+1}:\n" +
            "\n".join([f"Q{idx+1}: {row[f'q{idx+1}']}" for idx in range(20)]) + 
            (f"\nFree Text Response 1: {row['q21']}" if pd.notnull(row['q21']) else "") +
            (f"\nFree Text Response 2: {row['q22']}" if pd.notnull(row['q22']) else "")
            for i, row in stage_examples.iterrows()
        ]
    )

    # Structure the prompt messages for the LLM to generate a free-text response
    messages = [
        {
            "role": "user",
            "content": f"""
            {example_texts}
            
            Current Question: {question_text}
            If the question is question 1-20 (multiple choice), respond only with 1 number corresponding to your answer. If the question is 21 or 22 (free-response), please write a free response answer.
            Response:
            """
        }
    ]

    return messages

basemessage = [
    {
        "role": "system",
        "content": f"You are a person who has experienced the loss of a loved one in the military or veteran community. You are currently at the '{stage_name}' stage of grief: {description}. You are asked by the Tragedy Assistance Program for Survivors (TAPS) to fill out this survey based on your current stage of grief. Here are some example responses from others in this stage to guide your answer."
    },
]


In [22]:
import time
import os
import pandas as pd
from predictionguard import PredictionGuard

# Set up API key
os.environ["PREDICTIONGUARD_API_KEY"] = "Oq62vYfSJRwjnFQcUnJy5PM3SRVejYtJCXWSxnfv"
client = PredictionGuard()

responses_df = pd.DataFrame(columns=["Id", "label", "q21", "q22"])

for run_id in range(50):
    for stage, description in JourneyMap.items():
        stage_number = int(stage.split('.')[0])
        stagename = stage.split('.', 1)[1]
        responses = {"Id": f"{run_id}_{stagename}", 
                     "label": stage_number}
        
        # Generate response for q21
        question_text_q21 = SurveyQuestions[20]["question"]
        messages_q21 = specific_prompt_free_response(
            stage_number=stage_number,
            stage_name=stagename,
            description=description,
            question_text=question_text_q21,
            data=data
        )
        
        result_q21 = client.chat.completions.create(
            model="neural-chat-7b-v3-3",
            messages=messages_q21, 
            temperature=1, 
            max_tokens=300  # Higher token limit for detailed response
        )
        responses["q21"] = result_q21["choices"][0]["message"]["content"].strip()
        time.sleep(1)  # Small delay between requests to avoid rate limits
        
        # Generate response for q22
        question_text_q22 = SurveyQuestions[21]["question"]
        messages_q22 = specific_prompt_free_response(
            stage_number=stage_number,
            stage_name=stagename,
            description=description,
            question_text=question_text_q22,
            data=data
        )
        print(messages_q22)
        result_q22 = client.chat.completions.create(
            model="neural-chat-7b-v3-3",
            messages=messages_q22, 
            temperature=1, 
            max_tokens=300
        )
        responses["q22"] = result_q22["choices"][0]["message"]["content"].strip()
        time.sleep(1)
        
        # Append the generated responses to the DataFrame
        responses_df = pd.concat([responses_df, pd.DataFrame([responses])], ignore_index=True)

# Save or use responses_df as needed


[{'role': 'system', 'content': "You are a person who has experienced the loss of a loved one in the military or veteran community. You are currently at the ' Immediate Grief, Shock, and Emotion' stage of grief: Overwhelmed, loss of purpose; shock and trauma emotions (isolation) present and challenging to understand. Individuals may struggle to deal with family responsibilities alone. Surviving Child: Feeling disconnected without guidance and attention from grieving adults.. You are asked by the Tragedy Assistance Program for Survivors (TAPS) to fill out this survey based on your current stage of grief. Here are some example responses from others in this stage to guide your answer."}, {'role': 'user', 'content': '\n            Example 25:\nQ1: 1\nQ2: 2.0\nQ3: 0\nQ4: 1\nQ5: 2\nQ6: 0\nQ7: 3\nQ8: 4\nQ9: 1\nQ10: 0\nQ11: 3\nQ12: 0\nQ13: 1\nQ14: 3\nQ15: 1\nQ16: 1\nQ17: 0\nQ18: 0\nQ19: 0\nQ20: 0\nFree Text Response 2: "As a surviving child, the TAPS Seminar proved to be a valuable experience i

In [20]:
responses_df

,Id,label,q21,q22
0,"0_ Immediate Grief, Shock, and Emotion",1,My favorite moment of the weekend was particip...,"As a surviving child, attending the TAPS Semin..."
1,0_ Navigating Family Relationships,2,My favorite moment during the weekend seminar ...,As someone who has attended the TAPS Seminar d...
2,0_ Learning to Process Grief,3,"For me, a standout moment during the weekend w...","En el Seminario TAPS, encontré un ambiente seg..."
3,0_ Moments That Matter,4,One of my favorite moments from the weekend wa...,It can be challenging to deal with the loss of...
4,"0_ Feeling Immersed, Connected & Seen",5,"During the weekend TAPS seminar, I found mysel...",Los talleres durante el seminario TAPS fueron ...
...,...,...,...,...
295,49_ Navigating Family Relationships,2,My favorite moment from the weekend seminar wi...,The TAPS Seminar proved to be a vital aspect o...
296,49_ Learning to Process Grief,3,My favorite moment during the weekend at TAPS ...,As a surviving child attending the TAPS Semina...
297,49_ Moments That Matter,4,One of my favorite moments from the weekend wa...,The TAPS Seminar I attended allowed me to bett...
298,"49_ Feeling Immersed, Connected & Seen",5,"During the weekend seminar with TAPS, my favor...","During the TAPS seminar, I connected with othe..."


In [21]:
responses_df.to_csv('jomama.csv', index=False)